In [3]:
import pytesseract
from pytesseract import Output
import pandas as pd
pytesseract.pytesseract.tesseract_cmd=r"C:\\Program Files\\Tesseract-OCR\\tesseract.exe"

import numpy as np
import cv2 # OpenCV biblioteka
import matplotlib
import matplotlib.pyplot as plt  
# iscrtavanje slika i grafika unutar samog browsera
%matplotlib inline 
# prikaz vecih slika 
matplotlib.rcParams['figure.figsize'] = 16,14

In [4]:
custom_config = r'-c preserve_interword_spaces=1 --oem 1 --psm 4'

In [5]:
def get_text_with_indentation(image, config):
    d = pytesseract.image_to_data(image, config=config, output_type=Output.DICT)
    df = pd.DataFrame(d)

    final_text = ''

    # clean up blanks
    df1 = df[(df.conf!='-1')&(df.text!=' ')&(df.text!='')]
    # sort blocks vertically
    sorted_blocks = df1.groupby('block_num').first().sort_values('top').index.tolist()
    for block in sorted_blocks:
        curr = df1[df1['block_num']==block]
        sel = curr[curr.text.str.len()>3]
        char_w = (sel.width/sel.text.str.len()).mean()
        prev_par, prev_line, prev_left = 0, 0, 0
        text = ''
        for ix, ln in curr.iterrows():
            # add new line when necessary
            if prev_par != ln['par_num']:
                text += '\n'
                prev_par = ln['par_num']
                prev_line = ln['line_num']
                prev_left = 0
            elif prev_line != ln['line_num']:
                text += '\n'
                prev_line = ln['line_num']
                prev_left = 0

            added = 0  # num of spaces that should be added
            if ln['left']/char_w > prev_left + 1:
                added = int((ln['left'])/char_w) - prev_left
                text += ' ' * added 
            text += ln['text'] + ' '
            prev_left += len(ln['text']) + added + 1
        text += '\n'
        final_text += text

    return final_text

In [11]:
img = cv2.imread("../example2.png")
text = get_text_with_indentation(img, custom_config)
print(text)


  @GetMapping(path  = "/{id}") 
  public  ResponseEntity<?> findById(@PathVariable  Long id) { 

      try  { 
           Admin admin = adminService.findById(id) ; 
           return new ResponseEntity<>(adminMapper.toDto(admin),   HttpStatus.OK); 
       } catch (Exception e) { 
           return new ResponseEntity<>(HttpStatus  .NOT_FOUND) ; 

